#Primera versión paralela para el cálculo de componentes disjuntos

Se paralelizó el cálculo de componentes disjuntos aprovechando que en cálculo cada matriz independiente en el tiempo. 
Notas sobre el hardware: intel i7 2,6 GHz 3720QM 4 cores + hyperthreading 8GB RAM 1600

###Configuración inicial de Julia en paralelo

In [1]:
CPU_CORES

8

In [2]:
addprocs(CPU_CORES)

8-element Array{Any,1}:
 2
 3
 4
 5
 6
 7
 8
 9

In [4]:
cd("/Users/Nes/Desktop/NeuroCiencias/JuliaExperiments/Parallel-Neuroscience/ComponentesDisjuntos");

Carga los datos, aplica los filtros, laplacianos, etc.

In [12]:
include("init_parallel.jl")

0

###Configuración inicial de Julia en paralelo

La versión paralela se encuentra dentro del archivo obtenComponentesDisjuntosParallel.jl, básicamente es usar las macros @Parallel que mandan llamar a @spawn automáticamente para distribuir las rebanadas de tiempo entre los distintos procesos. Es importante usar @sync en el for para que el código espere que todos los workers terminen el loop. También se debe declarar cada función en cada worker con @everywhere.

La versión paralela tiene un speedup de 10x con respecto a la versión secuencial. Además de que los 8 workers procesan en paralelo cada rebanada de tiempo. Además, comando @Parallel, configura la rutina mapreduce para usar todos los núcleos, lo cual ayuda en las búsquedas de los índices de los fuentes y pozos. 

In [59]:
@time ObtenComponentesYEscribe(CSD,101,120)

elapsed time: 401.756996716 seconds (37209500428 bytes allocated, 69.73% gc time)


0

In [62]:
@time ObtenComponentesYEscribeP(CSD,101,120)

elapsed time: 38.634555157 seconds (1998970576 bytes allocated, 12.00% gc time)
